In [ ]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
def load_asd_data_with_pandas(data_path: str) -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        # print(f"Загружаем файл: {os.path.basename(file_path)}")
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


### По файлам


In [14]:
print("=== АНАЛИЗ ДУБЛИКАТОВ ПО ФАЙЛАМ ===\n")

parquet_files = glob.glob(os.path.join('./asd', "part-*.parquet"))
parquet_files.sort()

file_duplicates = []
for file_path in parquet_files:
    df = pd.read_parquet(file_path)
    file_name = os.path.basename(file_path)
    duplicates_in_file = df.drop(columns=['metadata']).duplicated().sum()
    file_duplicates.append({
        'file': file_name,
        'rows': len(df),
        'duplicates': duplicates_in_file,
        'duplicate_pct': duplicates_in_file / len(df) * 100 if len(df) > 0 else 0
    })

file_duplicates_df = pd.DataFrame(file_duplicates)
print("Дубликаты внутри каждого файла:")
print(file_duplicates_df.to_string(index=False))
print(f"\nОбщее количество дубликатов внутри файлов: {file_duplicates_df['duplicates'].sum()}")


=== АНАЛИЗ ДУБЛИКАТОВ ПО ФАЙЛАМ ===

Дубликаты внутри каждого файла:
                                                               file  rows  duplicates  duplicate_pct
part-00000-3a065afd-b2fa-4875-a6e5-911e95e3f86c-c000.snappy.parquet 61329        2777       4.528037
part-00001-74fdee0d-8448-4b0a-921c-f1f0ef356cdf-c000.snappy.parquet 60919        2740       4.497776
part-00002-634ededa-5adf-4170-93ba-2dac2bd74705-c000.snappy.parquet 63736        2550       4.000879
part-00003-427bc79e-0e40-4a8c-a2be-d0fbe09f03c0-c000.snappy.parquet 61152        2542       4.156855
part-00004-eb3dc336-995c-48bd-840f-49d411a89b8e-c000.snappy.parquet 60125        2936       4.883160
part-00005-846a5164-9ca8-4438-af5f-07ad5348f327-c000.snappy.parquet 61104        2816       4.608536
part-00006-b818506c-2926-406c-936b-66da5c9acdbc-c000.snappy.parquet 62158        3105       4.995334
part-00007-7ab8e466-84f8-43e0-8874-9c1bbf210d4e-c000.snappy.parquet 60256        2778       4.610329
part-00008-397aa529-5c

In [16]:
print("\n=== АНАЛИЗ ПЕРЕКРЫТИЙ МЕЖДУ ФАЙЛАМИ ===\n")

from itertools import combinations

file_unique_sets = {os.path.basename(fp): set(pd.read_parquet(fp).drop(columns=['metadata']).apply(tuple, axis=1)) for fp in parquet_files}

overlaps = [{'file1': f1, 'file2': f2, 'overlap_count': overlap, 'file1_size': len(s1), 'file2_size': len(s2), 
             'overlap_pct_file1': overlap / len(s1) * 100 if len(s1) > 0 else 0, 'overlap_pct_file2': overlap / len(s2) * 100 if len(s2) > 0 else 0}
            for (f1, s1), (f2, s2) in combinations(file_unique_sets.items(), 2) if (overlap := len(s1 & s2)) > 0]

if overlaps:
    overlaps_df = pd.DataFrame(overlaps)
    print("Найдены перекрытия между файлами:")
    print(overlaps_df.to_string(index=False))
    print(f"\nВсего пар файлов с перекрытиями: {len(overlaps)}")
    print(f"Общее количество перекрывающихся записей: {overlaps_df['overlap_count'].sum()}")
else:
    print("Перекрытий между файлами не обнаружено.")



=== АНАЛИЗ ПЕРЕКРЫТИЙ МЕЖДУ ФАЙЛАМИ ===

Перекрытий между файлами не обнаружено.


### По датасетам

In [ ]:
print("\n=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===\n")

dataset_analysis = []
for dataset in agab_df['dataset'].unique():
    dataset_df = agab_df[agab_df['dataset'] == dataset]
    duplicates = dataset_df.drop(columns=['metadata']).duplicated().sum()
    dataset_analysis.append({
        'dataset': dataset,
        'total_rows': len(dataset_df),
        'duplicates': duplicates,
        'duplicate_pct': duplicates / len(dataset_df) * 100 if len(dataset_df) > 0 else 0,
        'unique_rows': len(dataset_df) - duplicates
    })

dataset_analysis_df = pd.DataFrame(dataset_analysis).sort_values('duplicates', ascending=False)
print("Дубликаты по датасетам:")
print(dataset_analysis_df.to_string(index=False))



=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===

Дубликаты по датасетам:
                dataset  total_rows  duplicates  duplicate_pct  unique_rows
               covid-19       54625       27482      50.310297        27143
                    hiv       48008       24715      51.481003        23293
                 biomap        2725        1402      51.449541         1323
  structures-antibodies        2711        1170      43.157506         1541
                genbank        2989         717      23.987956         2272
  structures-nanobodies        1258         649      51.589825          609
                patents      217463         164       0.075415       217299
                   abbd      155853         132       0.084695       155721
               alphaseq      198703           4       0.002013       198699
flab_shanehsazzadeh2023         446           1       0.224215          445
               abdesign         672           0       0.000000          672
        flab_koenig2017

### Cтатистика

In [37]:
import json

print("=== Статистика дубликатов ===")
print(f"Всего строк: {len(agab_df)}")

# Создаем маску дубликатов один раз для эффективности
duplicates_mask = agab_df.drop(columns='metadata').duplicated()
duplicates_count = duplicates_mask.sum()

print(f"Дубликатов: {duplicates_count}")
print(f"Уникальных строк: {len(agab_df) - duplicates_count}")

# Дубликаты с учетом столбца metadata в JSON
print("\n=== Дубликаты с учетом metadata ===")

# Создаем временный DataFrame, преобразуя metadata в строку JSON
temp_df = agab_df.copy()
temp_df['metadata_json'] = temp_df['metadata'].apply(lambda x: json.dumps(x, sort_keys=True) if isinstance(x, dict) else str(x))

# Проверяем дубликаты, исключая столбец metadata (используем все столбцы кроме metadata)
columns_to_check = [col for col in temp_df.columns if col != 'metadata']
duplicates_with_metadata_mask = temp_df[columns_to_check].duplicated()
duplicates_with_metadata_count = duplicates_with_metadata_mask.sum()

print(f"Дубликатов с учетом metadata: {duplicates_with_metadata_count}")
print(f"Уникальных строк с учетом metadata: {len(agab_df) - duplicates_with_metadata_count}")

# Сравнительная статистика
print("\n=== Сравнительная статистика ===")
print(f"Разница в количестве дубликатов: {duplicates_count - duplicates_with_metadata_count}")

=== Статистика дубликатов ===
Всего строк: 1227083
Дубликатов: 56436
Уникальных строк: 1170647

=== Дубликаты с учетом metadata ===
Дубликатов с учетом metadata: 28602
Уникальных строк с учетом metadata: 1198481

=== Сравнительная статистика ===
Разница в количестве дубликатов: 27834
Metadata влияет на 27834 записей


In [40]:
# Распределение дубликатов по dataset (без учета metadata)
print("\n=== Распределение дубликатов по dataset (без учета metadata) ===")

# Добавляем флаг дубликатов в исходный DataFrame (без metadata)
agab_df['is_duplicate_no_metadata'] = duplicates_mask

# Группируем по dataset и считаем статистику
dataset_duplicates_no_metadata = agab_df.groupby('dataset').agg({
    'is_duplicate_no_metadata': ['count', 'sum', 'mean']
}).round(4)

# Переименовываем столбцы для удобства
dataset_duplicates_no_metadata.columns = ['total_rows', 'duplicates_count', 'duplicates_ratio']
dataset_duplicates_no_metadata = dataset_duplicates_no_metadata.reset_index()

# Сортируем по количеству дубликатов
dataset_duplicates_no_metadata = dataset_duplicates_no_metadata.sort_values('duplicates_count', ascending=False)

print("\nТоп-10 dataset по количеству дубликатов (без metadata):")
print(dataset_duplicates_no_metadata.head(10).to_string(index=False))

# Общая статистика по распределению
print(f"\nВсего dataset: {len(dataset_duplicates_no_metadata)}")
print(f"Dataset с дубликатами: {len(dataset_duplicates_no_metadata[dataset_duplicates_no_metadata['duplicates_count'] > 0])}")
print(f"Dataset без дубликатов: {len(dataset_duplicates_no_metadata[dataset_duplicates_no_metadata['duplicates_count'] == 0])}")



=== Распределение дубликатов по dataset (без учета metadata) ===

Топ-10 dataset по количеству дубликатов (без metadata):
                dataset  total_rows  duplicates_count  duplicates_ratio
               covid-19       54625             27482            0.5031
                    hiv       48008             24715            0.5148
                 biomap        2725              1402            0.5145
  structures-antibodies        2711              1170            0.4316
                genbank        2989               717            0.2399
  structures-nanobodies        1258               649            0.5159
                patents      217463               164            0.0008
                   abbd      155853               132            0.0008
               alphaseq      198703                 4            0.0000
flab_shanehsazzadeh2023         446                 1            0.0022

Всего dataset: 25
Dataset с дубликатами: 10
Dataset без дубликатов: 15

Максимальное

In [39]:
# Распределение дубликатов по dataset
print("\n=== Распределение дубликатов по dataset (с учетом metadata) ===")

# Добавляем флаг дубликатов в исходный DataFrame
agab_df['is_duplicate'] = duplicates_with_metadata_mask

# Группируем по dataset и считаем статистику
dataset_duplicates = agab_df.groupby('dataset').agg({
    'is_duplicate': ['count', 'sum', 'mean']
}).round(4)

# Переименовываем столбцы для удобства
dataset_duplicates.columns = ['total_rows', 'duplicates_count', 'duplicates_ratio']
dataset_duplicates = dataset_duplicates.reset_index()

# Сортируем по количеству дубликатов
dataset_duplicates = dataset_duplicates.sort_values('duplicates_count', ascending=False)

print("\nТоп-10 dataset по количеству дубликатов:")
print(dataset_duplicates.head(10).to_string(index=False))

# Общая статистика по распределению
print(f"\nВсего dataset: {len(dataset_duplicates)}")
print(f"Dataset с дубликатами: {len(dataset_duplicates[dataset_duplicates['duplicates_count'] > 0])}")
print(f"Dataset без дубликатов: {len(dataset_duplicates[dataset_duplicates['duplicates_count'] == 0])}")


=== Распределение дубликатов по dataset (с учетом metadata) ===

Топ-10 dataset по количеству дубликатов:
                dataset  total_rows  duplicates_count  duplicates_ratio
                    hiv       48008             24715            0.5148
                 biomap        2725              1402            0.5145
  structures-antibodies        2711               999            0.3685
  structures-nanobodies        1258               566            0.4499
                genbank        2989               480            0.1606
               covid-19       54625               303            0.0055
                   abbd      155853               132            0.0008
               alphaseq      198703                 4            0.0000
flab_shanehsazzadeh2023         446                 1            0.0022
               skempiv2         434                 0            0.0000

Всего dataset: 25
Dataset с дубликатами: 9
Dataset без дубликатов: 16
